In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,72052
2,Huelva,Confirmados PDIA 14 días,6793
3,Huelva,Tasa PDIA 14 días,"1323,7328760449755"
4,Huelva,Confirmados PDIA 7 días,2736
5,Huelva,Tasa PDIA 7 dias,"533,1566537404759"
6,Huelva,Total Confirmados,72287
7,Huelva,Curados,55279
8,Huelva,Fallecidos,441


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  72052.0


/tmp/ipykernel_16316/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  21050.0


/tmp/ipykernel_16316/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_16316/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_16316/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_16316/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 179 personas en los últimos 7 días 

Un positivo PCR cada 73 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,72052.0,6793.0,2736.0,513170.0,533.156654,1323.732876,420.0
Huelva-Costa,41719.0,3456.0,1401.0,289548.0,483.857599,1193.584483,195.0
Sierra de Huelva-Andévalo Central,9369.0,1893.0,827.0,67391.0,1227.166832,2808.980428,159.0
Huelva (capital),21050.0,1967.0,804.0,143837.0,558.966052,1367.520179,120.0
Condado-Campiña,20496.0,1435.0,505.0,156231.0,323.239306,918.511691,65.0
Valverde del Camino,2067.0,388.0,204.0,12750.0,1600.000000,3043.137255,52.0
Aracena,1361.0,186.0,103.0,8255.0,1247.728649,2253.179891,21.0
Lepe,3907.0,220.0,99.0,27880.0,355.093257,789.096126,16.0
Isla Cristina,3910.0,216.0,85.0,21393.0,397.326228,1009.676062,14.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),452.0,107.0,35.0,2327.0,1504.082510,4598.195101,3.0
Cortegana,708.0,207.0,81.0,4602.0,1760.104302,4498.044329,12.0
Jabugo,297.0,95.0,34.0,2260.0,1504.424779,4203.539823,5.0
Rosal de la Frontera,248.0,68.0,25.0,1697.0,1473.187979,4007.071302,4.0
Nerva,662.0,193.0,74.0,5169.0,1431.611530,3733.797640,14.0
Marines (Los),41.0,14.0,6.0,399.0,1503.759398,3508.771930,1.0
Valdelarco,13.0,8.0,4.0,231.0,1731.601732,3463.203463,0.0
Puerto Moral,38.0,9.0,4.0,278.0,1438.848921,3237.410072,1.0
Zarza-Perrunal (La),107.0,40.0,21.0,1253.0,1675.977654,3192.338388,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Fuenteheridos,81.0,13.0,1.0,709.0,141.043724,1833.568406,0.0,0.076923
Zalamea la Real,323.0,65.0,13.0,3054.0,425.671251,2128.356254,3.0,0.200000
Paymogo,121.0,10.0,2.0,1162.0,172.117040,860.585198,1.0,0.200000
Cumbres Mayores,299.0,47.0,10.0,1749.0,571.755289,2687.249857,3.0,0.212766
Gibraleón,1502.0,163.0,41.0,12737.0,321.896836,1279.736202,3.0,0.251534
Cartaya,3100.0,236.0,71.0,20083.0,353.532839,1175.123239,13.0,0.300847
Cerro de Andévalo (El),452.0,107.0,35.0,2327.0,1504.082510,4598.195101,3.0,0.327103
Niebla,607.0,61.0,20.0,4116.0,485.908649,1482.021380,0.0,0.327869
Rociana del Condado,927.0,45.0,15.0,7939.0,188.940673,566.822018,1.0,0.333333
